In [36]:
import os
import pandas as pd
import numpy as np
from io import StringIO
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [37]:
if torch.backends.mps.is_available():
    device = 'mps'
    mps_device = torch.device(device)
    torch.cuda.manual_seed_all(777)
    print(device)
else:
    print ("MPS device not found.")

torch.manual_seed(777)

mps


# 데이터 전처리

In [38]:
file_dir = './태양풍/'
file_names = [f'ace_{year}.csv' for year in range(1999, 2014)]

In [39]:
# 모든 파일을 읽어와서 데이터프레임 리스트에 저장
ace_data_list = [] # 비어있는 데이터 프레임 리스트 만들기
for file_name in file_names: # file_name 하나씩 조회
    file_path = os.path.join(file_dir, file_name) # file_dir랑 file_name 합쳐서 path 생성
    with open(file_path, 'r') as file: # read 모드로 열기
        lines = file.readlines() # 라인 읽어오기
        data_started = False 
        data_lines = [] # 비어있는 data_lines 생성
        for line in lines: # 읽은 라인들 하나씩 조회
            if "BEGIN DATA" in line: # 데이터 설명하는 row들이 끝나면
                data_started = True # data_started를 true로 바꿈
                continue
            if data_started: # data_started 이면 
                data_lines.append(line) # 라인 읽어서 data_lines에 붙이기
        # data_lines 리스트에 저장된 데이터를 데이터프레임으로 변환
        ace_data = pd.read_csv(StringIO('\n'.join(data_lines)), delim_whitespace=True, names=['year', 'doy', 'hr', 'min', 'Np', 'Tp', 'Vp', 'B_gsm_x', 'B_gsm_y', 'B_gsm_z', 'Bt'])
        # year, doy, hr, min 열을 하나의 datetime 형식으로 변환
        ace_data['datetime'] = pd.to_datetime(ace_data['year'].astype(str) + '-' + ace_data['doy'].astype(str) + ' ' + ace_data['hr'].astype(str) + ':' + ace_data['min'].astype(str), format='%Y-%j %H:%M')
        # 데이터프레임의 인덱스를 datetime 열로 설정
        ace_data = ace_data.set_index('datetime')
        # 특정 값들을 실제 결측값(NaN)으로 대체
        ace_data.replace([-9999.900, -9.9999e+03, -9999.90, -9999.9004], np.nan, inplace=True)
        # 전처리가 완료된 데이터프레임을 리스트에 추가
        ace_data_list.append(ace_data)

/var/folders/jq/ghsw3hz1139068l6f7s904y40000gn/T/ipykernel_4013/785987786.py:16: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  ace_data = pd.read_csv(StringIO('\n'.join(data_lines)), delim_whitespace=True, names=['year', 'doy', 'hr', 'min', 'Np', 'Tp', 'Vp', 'B_gsm_x', 'B_gsm_y', 'B_gsm_z', 'Bt'])
/var/folders/jq/ghsw3hz1139068l6f7s904y40000gn/T/ipykernel_4013/785987786.py:16: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  ace_data = pd.read_csv(StringIO('\n'.join(data_lines)), delim_whitespace=True, names=['year', 'doy', 'hr', 'min', 'Np', 'Tp', 'Vp', 'B_gsm_x', 'B_gsm_y', 'B_gsm_z', 'Bt'])
/var/folders/jq/ghsw3hz1139068l6f7s904y40000gn/T/ipykernel_4013/785987786.py:16: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep

In [40]:
# 모든 데이터프레임을 하나로 결합
ace_data_combined = pd.concat(ace_data_list)

In [41]:
from scipy.stats import norm

# 결측값이 아닌 데이터의 분포 추정
def estimate_distribution(column):
    non_na_data = column.dropna()
    mu, std = norm.fit(non_na_data)
    return mu, std

# 결측값을 추정된 분포에서 샘플링한 값으로 대체
def fill_missing_with_distribution(df):
    filled_df = df.copy() # 받은 데이터 복사본 만들기
    for column in df.columns: # column 하나씩 조회
        mu, std = estimate_distribution(df[column]) # 평균, 표준편차 구하기
        missing_mask = df[column].isna() # 결측치의 위치 표시
        filled_values = norm.rvs(loc=mu, scale=std, size=missing_mask.sum()) # 추정된 분포에서 샘플링
        filled_df.loc[missing_mask, column] = filled_values # 채워 넣기
    return filled_df

In [42]:
# 결측치 대체
ace_data_combined= ace_data_combined.drop(columns=['year', 'doy', 'hr', 'min'])
ace_filled = fill_missing_with_distribution(ace_data_combined)

In [43]:
ace_filled.to_csv('./cnn_data/pp_ace.csv')

In [44]:
# 지자기 교란 지수 데이터 불러오기 및 전처리
geomagnetic_data = pd.read_csv('./지자기교란 지수.csv')

# 'date' 열을 datetime 형식으로 변환
geomagnetic_data['date'] = pd.to_datetime(geomagnetic_data['date'])

In [45]:
# 지자기 교란 지수를 3시간 간격으로 전환
geomagnetic_long = geomagnetic_data.melt(id_vars=['date'], var_name='hour', value_name='Kp')
geomagnetic_long['hour'] = geomagnetic_long['hour'].str.extract('(\d+)').astype(int)
geomagnetic_long['datetime'] = geomagnetic_long.apply(lambda row: row['date'] + pd.Timedelta(hours=row['hour']), axis=1)

# datetime을 index로 사용
geomagnetic_long = geomagnetic_long.set_index('datetime')
# datetime 기준으로 sort
geomagnetic_long.sort_values(by='datetime', inplace=True)
geomagnetic_long = geomagnetic_long.drop(columns=['date', 'hour'])
geomagnetic_long.to_csv('./cnn_data/geomagnetic_long.csv')

# Spectogram 생성

In [71]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
from scipy.signal import spectrogram

In [72]:
import os
from scipy.signal import spectrogram
from scipy.stats import norm
from io import StringIO

In [92]:
# 각 컬럼별 스펙트로그램 생성 함수
def create_spectrogram(data, column_name, sr=60, n_fft=48, hop_length=10, n_mels=24):
    data_series = data[column_name].values
    S = librosa.feature.melspectrogram(y=data_series, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    S_db = librosa.power_to_db(S, ref=np.max)
    # 크기 조정
    S_db = librosa.util.fix_length(S_db, size=24, axis=1)
    return S_db

In [95]:
# 3시간 단위로 데이터를 나누고 각 구간별로 3D 스펙트로그램 생성 및 파일로 저장
def generate_spectrograms_for_intervals(df, interval='3H', output_dir='./3d_spectrograms/'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    interval_data = df.resample(interval)
    for start_time, group in interval_data:
        spectrograms = []
        for column in df.columns:
            if not group[column].isnull().all():
                spectrogram = create_spectrogram(group, column)
                spectrograms.append(spectrogram)
        
        # 3D 스펙트로그램 생성
        if spectrograms:
            spectrograms = np.array(spectrograms)
            #print(spectrograms.shape[1], spectrograms.shape[2])
            # Ensure all spectrograms have the same size
            spectrograms_resized = np.array([librosa.util.fix_length(s, size=24, axis=1) for s in spectrograms])
            spectrograms_resized = np.array([librosa.util.fix_length(s, size=24, axis=0) for s in spectrograms_resized])
            np.save(os.path.join(output_dir, f'spectrogram_{start_time.strftime("%Y-%m-%d_%H-%M-%S")}.npy'), spectrograms_resized)


In [97]:
generate_spectrograms_for_intervals(ace_filled, output_dir='./cnn_data/spectogram')

/var/folders/jq/ghsw3hz1139068l6f7s904y40000gn/T/ipykernel_4013/257439750.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  interval_data = df.resample(interval)


# CNN

In [151]:
# 데이터 로드 함수
def load_data(data_dir):
    X = []
    y = []
    indices = []
    for i, file in enumerate(os.listdir(data_dir)):
        if file.endswith('.npy'):
            file_path = os.path.join(data_dir, file)
            spectrogram = np.load(file_path)
            X.append(spectrogram)
            # 파일 이름에서 datetime 추출
            datetime_str = file.split('_')[1] + '_' + file.split('_')[2].split('.')[0]
            datetime_obj = pd.to_datetime(datetime_str, format='%Y-%m-%d_%H-%M-%S')
            kp_value = geomagnetic_long.loc[datetime_obj]['Kp']
            y.append(kp_value)
            indices.append(datetime_obj.year)  # 연도를 인덱스에 추가
    return np.array(X), np.array(y), np.array(indices)

In [168]:
# 데이터 로드
X, y, indices = load_data('./cnn_data/spectogram')

In [169]:
# 각 컬럼별로 정규화
X = X / np.max(X, axis=0)

/var/folders/jq/ghsw3hz1139068l6f7s904y40000gn/T/ipykernel_4013/116763702.py:2: RuntimeWarning: divide by zero encountered in divide
  X = X / np.max(X, axis=0)
/var/folders/jq/ghsw3hz1139068l6f7s904y40000gn/T/ipykernel_4013/116763702.py:2: RuntimeWarning: invalid value encountered in divide
  X = X / np.max(X, axis=0)


In [170]:
# 날짜별로 학습 데이터와 테스트 데이터로 분할
train_indices = np.where(indices < 2013)[0]
test_indices = np.where(indices == 2013)[0]

X_train, y_train = X[train_indices], y[train_indices]
X_test, y_test = X[test_indices], y[test_indices]

In [171]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(40912, 7, 24, 24)
(40912,)
(2920, 7, 24, 24)
(2920,)


In [172]:
# PyTorch Tensor로 변환
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long) # 정수형으로 변환
y_test = torch.tensor(y_test, dtype=torch.long) # 정수형으로 변환

In [173]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


torch.Size([40912, 7, 24, 24])
torch.Size([2920, 7, 24, 24])
torch.Size([40912])
torch.Size([2920])


In [174]:
# TensorDataset 및 DataLoader 설정
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [175]:
# 클래스 수 계산
num_classes = len(np.unique(y_train))

In [176]:
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(7, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 3 * 3, 256)  # 3x3은 입력 이미지 크기 24x24이 MaxPooling을 두 번 거친 후의 크기
        self.fc2 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.7)

        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 3 * 3)  # Flatten the output
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [177]:
model = CNNModel(num_classes).to(device)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0005)

In [ ]:
# 모델 학습
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs=inputs.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


In [163]:
from sklearn.metrics import mean_squared_error
# 모델 평가
model.eval()
correct = 0
total = 0
all_labels = []
all_predictions = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs=inputs.to(device)
        labels=labels.to(device)
        outputs = model(inputs.float())  # ensure inputs are float32
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

rmse = np.sqrt(mean_squared_error(all_labels, all_predictions))
print(f'Accuracy: {100 * correct / total}%')
print(f'RMSE: {rmse}')

Accuracy: 42.43150684931507%
RMSE: 1.0549946438869426
